Source publication for dataset: 
https://www.nature.com/articles/s41467-018-02866-0

In [1]:
import anndata as ad
import numpy as np
import pandas as pd
from copy import deepcopy
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
n_cells = 456
n_genes = 100
PWscores = np.zeros((n_cells, n_genes, n_genes))

for i in range(n_cells):
    path = "./../data/TEsmESC/geneXgene{i}.csv".format(i=(i+1))
    df_temp = pd.read_csv(path)
    PWscores[i, :, :] = np.copy(df_temp.values)
    

# Visualizing TE scores across cells (over time)

In [3]:
# heat_fig = sn.heatmap(PWscores[0, :, :], square=True)
    # as_file = heat_fig.get_figure()
    # path = "./../out/cell{i}.png".format(i = i + 1)
    # as_file.savefig(path)
    # ax.tick_params(left=False, bottom=False)

# The informative gene pathway extraction algorithm
(For now implemented as just an easy intuitive iterative solution, maybe later will attempt a numpy/C++ acceleration!)

In [4]:
# TODO: right now it is just greedy bfs, but I want paths to be able to branch and for multiple nodes to all connect to a common node
# eg. if you get 3-> 59, 18 -> 59 that outta be shown as 3&18 -> 59 (somehow)
"""
TODO: just build the incidence matrix as you go? each iteration adds a new set 'layer' of edges
--> see notes !
implement this proper (will write another function)
"""
def get_hyperedge_set(cell_index):

    cell_nw = PWscores[cell_index, :, :]
    
    # init
    paths = []
    max_itrs = n_genes
    itrs = 0
    tolerance = np.max(cell_nw) * 0.2 # all scores within whatever % of the max
    max_path_len = 0
    max_path_len_prev = -1

    for i in range(n_genes): #row is the predecessor node
        index_max = np.argmax(cell_nw[i, :]) # index of max column is the successor (incident) node
        
        if cell_nw[i, index_max] > tolerance:
            paths.append([i, index_max])

    # update
    while (max_itrs > itrs) and (max_path_len > max_path_len_prev):
        max_path_len_prev = max_path_len
        new_paths = deepcopy(paths) 
        for p in range(len(paths)):
            path = paths[p]
            if len(path) >= max_path_len:
                row = path[-1]
                index_max = np.argmax(cell_nw[row, :])
                
                if (cell_nw[row, index_max] > tolerance) and (index_max not in new_paths[p]):
                    new_paths[p].append(index_max)

        max_path_len = max( len(x) for x in new_paths )
        itrs += 1
        paths = deepcopy(new_paths)
    
    return paths

# Creation of the hypergraph

Note: a simple canonical edge has the form of tail->head

In [5]:
"""
Merges tails with common heads to identify all the multi-arity relations to represent as hedges and updates the supplied incidence matrix with these final hedges.
_edges: a list of tuples (tail_set, head_set) of all the initial single-tailed edges (note: they are sets, not lists!)
B: a list of n_gene-sized integer numpy arrays to represent the incidence matrix to update.
TODO: Test and be especially make sure Python's memory system isn't doing any weird reference copying and producing the wrong stuff!
Note, all set operations return copies, so this SHOULD be okay.
"""
def merge_edges(_edges, B):
    
    # init
    edges = deepcopy(_edges)
    m = len(edges)

    # Stop when there's only a single edge left (not possible to merge anything else)
    while (m >= 2):
        
        new_edges = [] 
        for i in range(0, m, 1):
            e1 = edges[i]
            for j in range(i + 1, m, 1):
                e2 = edges[j]
                common_heads = e1[1].intersection(e2[1])
                if (len(common_heads) > 0):
                    
                    # The new edge
                    new_tails = e1[0].union(e2[0]) 
                    new_edge = ([new_tails, common_heads])
                    new_edges.append(new_edge)

                    # Updating old edges (their head sets)
                    e1[1] = set(e1[1] - common_heads)
                    e2[1] = set(e2[1] - common_heads)

        
        # TODO: weight the edges?
        # Add all edges with non-null head sets to the incidence matrix
        for edge in edges:
            col = np.zeros(n_genes)
            if len(edge[1]) > 0:
                for tail in edge[0]:
                    col[tail] = 1 #TODO: TESTING
                for head in edge[1]:
                    col[head] = 1

                B.append(np.copy(col))
        
        edges = deepcopy(new_edges)
        m = len(edges)

In [6]:
""" 
Creates a hypergraph, represented using an incidence matrix, from the given cell's PW gene network. 
cell_index: the index into the PWscores array for the desired cell.
p: percentage parameter to use in tolerance calculation; decides which relationships relative to the maximum PW TE. 
returns: B the incidence matrix of the constructed hypergraph. 
"""
def construct_hyper_graph(cell_index, p):
    # init
    unique_genes = set()
    cell_nw = PWscores[cell_index, :, :]

    # The incidence matrix (might use different data structure)
    B = []

    # all scores within whatever % of the max
    tolerance = np.max(cell_nw) * p

    tails = set(np.arange(start=0, stop=n_genes, step=1))

    # debug var
    count = 0

    # TODO: TEST IT ALL 
    while (len(tails) > 0):
        edges = []
        new_tails = set()
        heads_to_remember = set()

        for tail in tails:
            heads = set(np.flatnonzero(cell_nw[tail, :] > tolerance))
            
            # Termination condition: once there's no new heads added this never gets hit, no new tails get added and the tail set becomes null at the end of while itr
            if (len(heads) > 0) and (heads.isdisjoint(unique_genes)):
                
                count += 1
                
                for head in heads:
                    # error handling
                    if (cell_nw[tail, head] <= tolerance):
                        print("BIG BUG") # TODO: keep this error cond here and handle it better!

                    # update the next-level tails
                    new_tails.add(head)
                    
                    # TODO: maybe make the set of all heads for this level here and wait till the end of the iteration to deep copy it over?
                    heads_to_remember.add(head)
                    
                # update the edge set for this hgraph level
                tail_as_set = set()
                tail_as_set.add(tail)
                edges.append([tail_as_set, heads])
        
        # update the unique heads set
        # for e in edges:
        #     for h in e[1]:
        #         unique_genes.add(h)
        for h in heads_to_remember:
            unique_genes.add(h)

        merge_edges(edges, B)
        tails = set(deepcopy(new_tails))
    
    return B



In [7]:
hedges = []
# TODO: Test proper via some contrived examples
for i in range(n_cells):
    G = construct_hyper_graph(i, 0.7)
    if (len(G) > 0):
        hedges.append(np.transpose(np.array(G)))


# Combining the edges into one big hypergraph for clustering

In [8]:
H = np.concatenate(hedges, axis=1)
H.shape

(100, 2065)

# Spectral clustering to determine pathways

In [9]:


# # TODO: calculate using Adjacency and Degree matrices?
# L = H @ np.transpose(H)
# eigvals, eigvecs = np.linalg.eig(L)

# # testing to make sure all eigenvalues are real and non-negative
# if (np.sum(np.iscomplex(eigvals)) > 0) or (np.sum(eigvals < 0) > 0):
#     raise ValueError("Laplacian is not a positive semidefinite matrix")

# # TODO: test and debug this piece! sort based on eigenvalues
# vecs = eigvecs[:,np.argsort(eigvals)]
# vals = eigvals[np.argsort(eigvals)]

# # print(vals)
# # # TODO: this approach requires normalized Laplacian
# # k = vals.shape[0] - np.sum(vals  0)
# k = 4
# # TODO: Fit from the first non-zero eigenvalue? Need to do this?
# index = np.where(vals > 0)[0][0] 

# kmeans = KMeans(n_clusters=k)
# kmeans.fit(vecs[:,])
# np.unique(kmeans.labels_, return_counts=True) # clusters and the number of genes assigned to each cluster


In [10]:
"""
Spectral Clustering Proper
T = Dv**-0.5 * (H * W * De**-1 * H.T) * Dv**-0.5 TODO: maybe don't ignore hedge weights, so refactor and do all that proper? First this though.
Delta = I - T
L_normalized = 0.5 * (I - (Dvis @ A @ Dvis)) Dvis = inverse square root of vertex Degree matrix
A = adjacency matrix = H @ W @ H.T - Dv
Dv is a diagonal matrix containing vertex degrees, i.e. number of edges the vertex appears in (vertices x vertices) 
H = incidence matrix
W = diagonal matrix of hyperedge weights (hyperedges x hyperedges)
De = Diagonal matrix of hyperedge degrees, i.e. the number of vertices in each hyper edge (hedges x hedges) 
"""

# pruning TFs (because of how python works, it makes more sense to first figure out all the indices where to delete, then delete)
count = 0
to_prune = []
gene_labels = df_temp.columns.to_numpy()

for v in range(n_genes):
    deg_v = np.sum(H[v,:])
    if deg_v == 0:
        # remove the vertex, since it isn't involved in any hedges, therefore won't be in any pathways
        count += 1
        to_prune.append(v)

n_genes = n_genes - count

In [11]:

pruned_labels = np.delete(gene_labels, to_prune)
pruned_H = np.delete(H, to_prune, axis=0) # delete rows with indices in to_prune

# Checking it did its job
if np.intersect1d(pruned_labels, gene_labels[to_prune]).shape[0] != 0:
    raise ValueError("Pruning of background genes failed.")


In [12]:
# Creating the Diagonal vertex and edge degree matrix  
Dv = np.zeros((n_genes, n_genes))
De = np.zeros((pruned_H.shape[1], pruned_H.shape[1]))


for v in range(n_genes):
    deg_v = np.sum(pruned_H[v,:])
    if deg_v == 0:
        raise ValueError("Pruning of background genes failed.")
    else:
        Dv[v, v] = deg_v

for e in range(pruned_H.shape[1]):
    De[e, e] = np.sum(pruned_H[:, e])

# Weights for each hyperedge = number of vertices in that edge, just need W to keep that sum
# W = 

# Adjacency matrix of the hypergraph
A = (pruned_H @ pruned_H.T) - Dv

# Normalized Laplacian of the hypergraph 
Dvis = np.linalg.inv(np.sqrt(Dv)) # replace each diagonal entry with the reciprocal of its square root, i.e. di with 1/sqrt(di)
L_normal = np.eye(n_genes) - (Dvis @ pruned_H @ np.linalg.inv(De) @ (pruned_H.T) @ Dvis) # TODO: add W matrix later
L = Dv - A

In [13]:
"""
Sanity checking A, D, and L
Dv = number of hyperedges a vertex appears in
A = is the adjacency matrix: vertex x vertex, 1 if the edge exits (i.e. the two vertices are adjacent)
  = For a Hypergraph, that's a tensor, i.e. avoid 'em
L = Laplacian, classically defined as Dv - A; properties: all non-neg real eigenvals, every row sums to 0, always has a 0 eigenval with eigenvector of all constants
"""
eigvals, eigvecs = np.linalg.eig(L_normal) # L-norm look really good!
# eigvals[np.where(eigvals < 0.5)[0]]
# eigvals[eigvals < 0.3]
# eigvecs[55]
np.sum(eigvals < 0)
np.argmin(eigvals)
# eigvecs[:,0] are the eigenvecs the rows or columns of this matrix? They're the columns
np.sum(L_normal * eigvecs[:,0]) # that's pree much 0


-8.500145032286355e-16

In [20]:
# TODO: Time to cluster!

# sort eigenvals, then sort eigenvecs by the eigenvals
sorted_eigvals = np.sort(eigvals)
sorted_eigvecs = eigvecs[:,np.argsort(eigvals)] # sorts the columns

# TODO: gotta read up more on how this k means version works
k = 10
# TODO: Fit from the first non-zero eigenvalue? Need to do this?
index = np.where(vals > 0)[0][0] 

kmeans = KMeans(n_clusters=k)
kmeans.fit(vecs[:,])

# assign labels


# visualize the clusters


array([ 2.11721798e-02,  1.38993693e-02, -6.86490122e-03,  5.57677819e-02,
        9.35075824e-02, -3.99601165e-02, -9.49147884e-02,  6.37455604e-01,
        1.18984792e-02, -3.90418358e-02,  3.63638894e-02,  5.74964334e-04,
       -3.04356350e-02, -6.88835841e-01, -1.77071600e-02, -2.51695616e-02,
        1.23331230e-03, -4.09342677e-02, -6.35831316e-03,  1.08419182e-02,
        6.97520283e-03,  4.36746314e-03,  2.48797106e-03, -4.47106367e-03,
        3.89569572e-03, -1.15139070e-02, -1.42368518e-03,  1.31583950e-02,
        5.80380382e-02,  3.56794868e-03, -1.78712593e-03,  1.33670695e-03,
       -7.72215055e-03, -8.54178767e-04,  5.85760909e-02, -4.02597233e-03,
        2.65879916e-02,  8.44576395e-03, -1.09484160e-02, -6.23052400e-02,
       -9.43920889e-03,  1.44087263e-02,  1.98140253e-03,  1.73590587e-03,
       -3.46150660e-03,  4.80638440e-02,  7.86929532e-03, -8.30313711e-02,
       -2.42196090e-02,  3.37935692e-02,  5.70344493e-03,  5.88750805e-03,
        2.27820296e-01, -

In [21]:
sorted_eigvecs[:, -1]

array([ 2.11721798e-02,  1.38993693e-02, -6.86490122e-03,  5.57677819e-02,
        9.35075824e-02, -3.99601165e-02, -9.49147884e-02,  6.37455604e-01,
        1.18984792e-02, -3.90418358e-02,  3.63638894e-02,  5.74964334e-04,
       -3.04356350e-02, -6.88835841e-01, -1.77071600e-02, -2.51695616e-02,
        1.23331230e-03, -4.09342677e-02, -6.35831316e-03,  1.08419182e-02,
        6.97520283e-03,  4.36746314e-03,  2.48797106e-03, -4.47106367e-03,
        3.89569572e-03, -1.15139070e-02, -1.42368518e-03,  1.31583950e-02,
        5.80380382e-02,  3.56794868e-03, -1.78712593e-03,  1.33670695e-03,
       -7.72215055e-03, -8.54178767e-04,  5.85760909e-02, -4.02597233e-03,
        2.65879916e-02,  8.44576395e-03, -1.09484160e-02, -6.23052400e-02,
       -9.43920889e-03,  1.44087263e-02,  1.98140253e-03,  1.73590587e-03,
       -3.46150660e-03,  4.80638440e-02,  7.86929532e-03, -8.30313711e-02,
       -2.42196090e-02,  3.37935692e-02,  5.70344493e-03,  5.88750805e-03,
        2.27820296e-01, -

# Visualizing Eigenvalues for Hyperparameter Selection

In [15]:
# x = np.arange(eigvals.shape[0])
# y = np.flip(eigvals)
# plt.scatter(x, y)
# np.sum((eigvals > 0) and (eigvals < 5))
# nonzero_eigvals = eigvals[np.where(eigvals > 0)[0]]
# final_eigvals = nonzero_eigvals[np.where(nonzero_eigvals < 5)[0]]
# final_eigvals
# fiedler_val = np.where(eigvals > 0)[0][-1]
# fiedler_vec = eigvecs[:,fiedler_val] 
# fiedler_vec

array([-4.62736121e-04,  1.43412089e-01, -3.38885739e-02, -8.45916279e-02,
        2.16501394e-02,  5.71844298e-02, -9.98682743e-02,  5.80163531e-02,
        1.81195810e-01, -6.68441750e-02, -7.38993784e-02,  1.41321318e-01,
        2.42739312e-02,  6.35017863e-02,  7.36344354e-02, -5.45729489e-02,
        5.58404987e-02,  1.10501587e-02,  7.20370780e-02, -7.78206264e-02,
        3.19246967e-01, -9.84412544e-03, -1.03354881e-01,  6.57086561e-03,
       -7.20045662e-02,  4.14004926e-02, -2.23165727e-02, -3.22628083e-01,
       -9.04463870e-02, -4.26938684e-03, -2.40326234e-02, -5.75283604e-02,
        3.03606768e-02, -8.09847577e-03,  4.79136191e-03,  9.31445503e-02,
        9.62451271e-04,  8.81871615e-03, -1.19701263e-01, -1.81837739e-02,
       -5.83033030e-02,  3.57901821e-02,  1.92190934e-02,  2.68810379e-01,
        8.31259519e-02,  3.91983068e-03, -2.58258200e-01,  3.74066337e-02,
        2.71628271e-02,  1.21667172e-01,  1.91625628e-02,  3.04732871e-04,
        1.75004014e-02,  

In [16]:
# gene_labels = df_temp.columns.to_numpy()
# # gene_labels[0] == 'SOX2'
# # num_clusters = np.unique(kmeans.labels_).shape[0] #want a set for each cluster

# clusters = []
# for i in range(k):
#     genes = np.argwhere(kmeans.labels_ == i).flatten()
#     if genes.shape[0] > 1:
#         clusters.append(genes)

# for c in clusters:
#         print("here")
#         # print(gene_labels[c])

NameError: name 'k' is not defined

In [ ]:
# # # sn.histplot(eigs)
# # #456 vectors, each one has a set of values, I want a count for each unique value
# eigvals[eigvals < 1] = 0
# # eigvals = np.argwhere(eigvals > 10)
# bins, counts = np.unique(eigvals, return_counts=True) # tuple for each eigen value: the value and its count
# # # sn.histplot(x=hist[1], y=hist[0])
# # # plt.pyplot.hist(bins, weights=counts)
# plt.hist(bins[:], color = 'blue', edgecolor = 'black', weights=counts[:])
# plt.xlabel("eigenvalues")
# plt.ylabel("counts")